### Testing entire model training process for moving to NCI

In [7]:
# Autoreload 
%load_ext autoreload
%autoreload 2

from pathlib import Path
from phosphosite.dataset import psp_filtered


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# TODO: 
# combine positive example selection and negative example selection (i.e. creating the index dict)
# into one function

In [11]:
# Load index dict 
filepath = Path("./indexes_dict.json")

from phosphosite.utils.io import save_index_dict, load_index_dict

filepath = Path("./indexes_dict.json")
index_dict = load_index_dict(filepath=filepath)

index_dict

{'P30556': {'idx': tensor([334, 337, 341, 331, 335,  25, 318,  46, 214, 213, 291, 147,  53]),
  'y': tensor([1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.])},
 'P24903': {'idx': tensor([  2, 422, 429,  65,  22,  20, 107, 448, 286, 127, 290, 301,  80, 100,
           37, 457, 374, 389, 185, 371,  95]),
  'y': tensor([1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0.])},
 'A8K0S8': {'idx': tensor([ 79, 310, 284, 174, 347,  92,  12,  10, 177,   4]),
  'y': tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])},
 'P49716': {'idx': tensor([ 56, 166, 190, 255, 170, 147, 144,  66,  61,  52, 159,  87]),
  'y': tensor([1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.])},
 'O76003': {'idx': tensor([ 31, 116, 117, 119, 123, 182, 195, 108, 152, 300, 199,  93, 221]),
  'y': tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.])},
 'Q3SYB3': {'idx': tensor([ 10,  53, 117, 160,  99, 273,  95, 209, 195]),
  'y': tensor([1., 0., 0., 0., 0., 0., 0., 0., 0.]

In [10]:
index_dict

{'P30556': {'idx': [334,
   337,
   341,
   331,
   335,
   25,
   318,
   46,
   214,
   213,
   291,
   147,
   53],
  'y': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]},
 'P24903': {'idx': [2,
   422,
   429,
   65,
   22,
   20,
   107,
   448,
   286,
   127,
   290,
   301,
   80,
   100,
   37,
   457,
   374,
   389,
   185,
   371,
   95],
  'y': [1.0,
   1.0,
   1.0,
   1.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0]},
 'A8K0S8': {'idx': [79, 310, 284, 174, 347, 92, 12, 10, 177, 4],
  'y': [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]},
 'P49716': {'idx': [56, 166, 190, 255, 170, 147, 144, 66, 61, 52, 159, 87],
  'y': [1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]},
 'O76003': {'idx': [31,
   116,
   117,
   119,
   123,
   182,
   195,
   108,
   152,
   300,
   199,
   93,
   221],
  'y': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

Get labels

In [ ]:

from phosphosite.dataset.psp import load_psp_alternate
psp = load_psp_alternate(
    include_columns=["uniprot_id", "res", "pos", ]
)
psp
# Order by uniprot_id 
psp = psp.sort_values(by=["uniprot_id", "res", "pos"])
psp
# subtract 1 from pos to get index. 
psp["index"] = psp["pos"] - 1
# group by uniprot_id to get the list of phosphosites for each uniprot_id (positions)
psp = psp.groupby("uniprot_id").agg(list)
psp

psp_uniprot_ids_to_use = psp.index.tolist()

with open("psp_uniprot_ids_to_use.txt", "w") as f:
    f.write("\n".join(psp_uniprot_ids_to_use))


# turn dataframe into a dictionary
site_dict = psp.to_dict(orient="index", )
site_dict

position_dict = dict(zip(psp.index, psp["index"]))
position_dict 